# Predict purchasing probability for financial product - Apply trained model

### Introduction
-  Push calculations down to SAP HANA. 
-  Leverage the in-memory performance directly out of Python without data extraction.

### Documentation
-  SAP HANA Python Client API for Machine Learning Algorithms:   
   https://help.sap.com/doc/0172e3957b5946da85d3fde85ee8f33d/latest/en-US/html/hana_ml.html
-  SAP HANA Automated Predictive Library (APL):
   https://help.sap.com/viewer/cb31bd99d09747089754a0ba75067ed2/latest/en-US/5db34eae84ed44e49ec1f0f78dfb52a1.html
-  Dataset: https://archive.ics.uci.edu/ml/datasets/bank+marketing

### Create connection to SAP HANA

In [1]:
# Obtain logon details from the central Connection Management of SAP Data Intelligence
import hana_ml.dataframe as dataframe

# Instantiate connection object
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
conn = NotebookConnectionContext(connectionId='HANADB')

### Load model

In [2]:
from hana_ml.algorithms.apl.classification import AutoClassifier
apl_model_loaded = AutoClassifier(conn_context=conn)
apl_model_loaded.load_model(schema_name='TAXX', table_name='BANKMARKETING_MODEL_APL')

### Predict probability of purchase for a "new" person

In [3]:
# Create HANA dataframe for the table that holds the data for prediction
df_pushdown_new = conn.table("BANKMARKETING_TOPREDICT", schema="TAXX")
df_pushdown_new.head(5).collect()

,CLIENT_ID,AGE,JOB,MARITAL,EDUCATION,DEFAULT,BALANCE,HOUSING,LOAN,CONTACT,DAY,MONTH,POUTCOME
0,1.0,39.0,blue-collar,married,secondary,no,22458.0,yes,no,cellular,17.0,may,unknown
1,2.0,39.0,blue-collar,married,secondary,no,22458.0,yes,no,cellular,17.0,april,unknown


In [4]:
# Predict the probability
apl_model_loaded.extra_applyout_settings =  {'APL/ApplyProbability':'true'}
df_pushdown_predict = apl_model_loaded.predict(df_pushdown_new)
df_pushdown_predict.head(10).collect()

,CLIENT_ID,PREDICTED,PROBABILITY,Y,proba_rr_Y
0,1.0,no,0.938995,None,0.061005
1,2.0,no,0.926306,None,0.073694


In [5]:
# Write the prediction to a table
df_pushdown_predict.save(where=("TAXX", "BANKMARKETING_PREDICTED"), table_type="COLUMN")

### Close connection

In [6]:
conn.close()